In [9]:
import pandas as pd
import os
from find_vehicle_image_urls import *

In [10]:


search_results_df=compile_search_results_df()
search_results_df.head()


File search_results_alfa romeo_4c.csv.csv wrong columns, skipping
File search_results_porsche_taycan_1739322969.csv wrong columns, skipping
File search_results_porsche_taycan_1739323122.csv wrong columns, skipping
File search_results_ford_taurus_1739323840.csv wrong columns, skipping
File search_results_alfa romeo_4c.csv wrong columns, skipping
File search_results_hyundai_asgadgadhadhsfqtdhg.csv wrong columns, skipping
File search_results_ford_lightning_1739323705.csv wrong columns, skipping
File search_results_porsche_taycan_1739305424.csv wrong columns, skipping
File search_results_hyundai_ioniq5.csv.csv wrong columns, skipping
File search_results_alfa-romeo_4c.csv wrong columns, skipping


,listing_header,url,search_url,make,model,search_timestamp,search_metadata,filename,url_clean,vehicle_id
0,Used 2020 Honda Odyssey EX-L,https://www.autotrader.com/cars-for-sale/vehic...,https://www.autotrader.com/cars-for-sale/honda...,honda,odyssey,1739360198,"{'make': 'honda', 'model': 'odyssey'}",search_results_honda_odyssey_1739360198.csv,https://www.autotrader.com/cars-for-sale/vehic...,732504624
1,Used 2011 Honda Odyssey EX-L,https://www.autotrader.com/cars-for-sale/vehic...,https://www.autotrader.com/cars-for-sale/honda...,honda,odyssey,1739360198,"{'make': 'honda', 'model': 'odyssey'}",search_results_honda_odyssey_1739360198.csv,https://www.autotrader.com/cars-for-sale/vehic...,738409771
2,Used 2024 Honda Odyssey Sport,https://www.autotrader.com/cars-for-sale/vehic...,https://www.autotrader.com/cars-for-sale/honda...,honda,odyssey,1739360198,"{'make': 'honda', 'model': 'odyssey'}",search_results_honda_odyssey_1739360198.csv,https://www.autotrader.com/cars-for-sale/vehic...,736531071
3,Certified 2024 Honda Odyssey EX-L,https://www.autotrader.com/cars-for-sale/vehic...,https://www.autotrader.com/cars-for-sale/honda...,honda,odyssey,1739360198,"{'make': 'honda', 'model': 'odyssey'}",search_results_honda_odyssey_1739360198.csv,https://www.autotrader.com/cars-for-sale/vehic...,731560875
4,Used 2019 Honda Odyssey EX-L,https://www.autotrader.com/cars-for-sale/vehic...,https://www.autotrader.com/cars-for-sale/honda...,honda,odyssey,1739360198,"{'make': 'honda', 'model': 'odyssey'}",search_results_honda_odyssey_1739360198.csv,https://www.autotrader.com/cars-for-sale/vehic...,736627883


In [11]:
## let's learn how many vehicle urls are provided by the typical search
print(search_results_df.groupby(['filename']).size().describe())
np.mean(search_results_df.groupby(['filename']).size() >=20)

count    289.000000
mean      27.830450
std       10.550917
min        3.000000
25%       29.000000
50%       30.000000
75%       30.000000
max      150.000000
dtype: float64


0.903114186851211

In [12]:
## dedup (devastating)
n_before = len(search_results_df)
search_results_df=search_results_df.drop_duplicates(subset=['url'])
n_after = len(search_results_df)
message =f'after deduplicating I go from {n_before} to {n_after} records'
print(message)


## let's learn how many vehicle urls are provided by the typical search

print(search_results_df.groupby(['filename']).size().describe())
np.mean(search_results_df.groupby(['filename']).size() >=20)

after deduplicating I go from 8043 to 5338 records
count    289.000000
mean      18.470588
std       14.961293
min        1.000000
25%        4.000000
50%       29.000000
75%       30.000000
max      150.000000
dtype: float64


0.5363321799307958

In [13]:

# df, _ = process_vehicle_webpage(search_results_df['url'][1])

In [14]:
i=2

search_results_df.loc[i, ]
vehicle_id = search_results_df.loc[i, 'vehicle_id']
filepath=f'{parent_directory_url_csvs}{vehicle_id}.csv'
filepath
already_exist = os.path.exists(filepath)
filepath, already_exist


if already_exist:
    print(f'file {filepath} already exists, skipping')
else:
    df, _ = process_vehicle_webpage(search_results_df.loc[i, 'url_clean'])


file /Users/levgolod/Projects/car_classifier/data/autotrader/vehicle_metadata/736531071.csv already exists, skipping


In [15]:
!ls -lsh ~/Projects/car_classifier/data/autotrader/vehicle_metadata/738409771.csv

24 -rw-r--r--@ 1 levgolod  staff   9.9K Feb 12 09:09 /Users/levgolod/Projects/car_classifier/data/autotrader/vehicle_metadata/738409771.csv


In [16]:
search_results_df = search_results_df.drop_duplicates(subset=['url_clean'])
search_results_df = search_results_df.sample(frac=1)
search_results_df.reset_index(drop=True,inplace=True)
min_sleep=90
sleep = min_sleep+1
time.sleep(sleep)

for i in search_results_df.index:

    search_results_df.loc[i, ]
    vehicle_id = search_results_df.loc[i, 'vehicle_id']
    url_clean = search_results_df.loc[i, 'url_clean']
    filepath=f'{parent_directory_url_csvs}{vehicle_id}.csv'
    already_exist = os.path.exists(filepath)

    if already_exist:
        print(f'file {filepath} already exists, skipping')
        continue

    message = f'''
###########################################################
### Begin processing {i, vehicle_id} | {dt.datetime.now().replace(microsecond=0)}
###########################################################
    '''
    print(message) # second

    try:
        with timethis(url_clean):
            df, _ = process_vehicle_webpage(url=url_clean, quit=True)

    except Exception as e:
        error_message = traceback.format_exc()
        print(f"notebook 03 - [{url_clean}] Error Traceback:\n", error_message)
        df = pd.DataFrame()

    if len(df)==0:
        sleep += min_sleep
        print(f'due to the cruelty of the gods, sleep param set to {sleep}')
    else:
        sleep -= min_sleep
        sleep = max(sleep, min_sleep)

        # new_sleep = max(min_sleep, sleep-min_sleep)
        # if new_sleep != sleep:
        #     print(f'sleep param set to {new_sleep}')
        #     sleep = new_sleep

    print(f'sleeping {sleep} seconds - starting {dt.datetime.now().replace(second=0, microsecond=0)}')
    time.sleep(sleep)



file /Users/levgolod/Projects/car_classifier/data/autotrader/vehicle_metadata/735444152.csv already exists, skipping

###########################################################
### Begin processing (1, '738076630') | 2025-02-16 18:54:37
###########################################################
    
single vehicle listing [https://www.autotrader.com/cars-for-sale/vehicle/738076630]- webpage initiated

year_make_model - success
list_price - success
vin - success
listing_detail - success
listing_narrative - success
header image - success
view_all_media clicked successfully
the futility is unbearable 21 5
get_image_urls_from_view_all_media_button - success - 25
found 26 images
                                                                   0
vehicle_image_url  https://images.autotrader.com/scaler/500/375/h...
vehicle_id                                                 738076630
url                https://www.autotrader.com/cars-for-sale/vehic...
vin                                    

KeyboardInterrupt: 

In [ ]:
# single vehicle listing [https://www.autotrader.com/cars-for-sale/vehicle/730197477]- webpage initiated
#     only  4 images?

# socket.timeout: timed out
#
# The above exception was the direct cause of the following exception:
#
# Traceback (most recent call last):
#   File "/Users/levgolod/Projects/car_classifier/find_vehicle_image_urls.py", line 273, in process_vehicle_webpage
#     driver.get(url_cleaned)
#   File "/Users/levgolod/Projects/venv3/lib/python3.8/site-packages/selenium/webdriver/remote/webdriver.py", line 393, in get
#     self.execute(Command.GET, {"url": url})
#   File "/Users/levgolod/Projects/venv3/lib/python3.8/site-packages/selenium/webdriver/remote/webdriver.py", line 382, in execute
#     response = self.command_executor.execute(driver_command, params)
#   File "/Users/levgolod/Projects/venv3/lib/python3.8/site-packages/selenium/webdriver/remote/remote_connection.py", line 404, in execute

In [ ]:
process_vehicle_webpage [https://www.autotrader.com/cars-for-sale/vehicle/722919380] Error Traceback:
 Traceback (most recent call last):
  File "/Users/levgolod/Projects/car_classifier/find_vehicle_image_urls.py", line 433, in process_vehicle_webpage
    image_urls = get_image_urls_from_view_all_media_button(driver)
  File "/Users/levgolod/Projects/car_classifier/find_vehicle_image_urls.py", line 309, in get_image_urls_from_view_all_media_button
    driver.execute_script(f"arguments[0].scrollTop += {scroll_distance};", scroll_panel)
  File "/Users/levgolod/Projects/venv3/lib/python3.8/site-packages/selenium/webdriver/remote/webdriver.py", line 444, in execute_script
    return self.execute(command, {"script": script, "args": converted_args})["value"]
  File "/Users/levgolod/Projects/venv3/lib/python3.8/site-packages/selenium/webdriver/remote/webdriver.py", line 384, in execute
    self.error_handler.check_response(response)
  File "/Users/levgolod/Projects/venv3/lib/python3.8/site-packages/selenium/webdriver/remote/errorhandler.py", line 232, in check_response

In [ ]:
# https://www.autotrader.com/cars-for-sale/vehicle/737396206
# only 1 image captured?
# in these instances should maybe try again